# Advanced Operating Systems: Lab 1 - Getting Started with Kernel Tracing

This first lab, worth only 10% of your overall marks, is to teach some baseline skills in using DTrace and Jupyter to do OS tracing, analysis, and data presentation. You are encouraged to explore the assignment using this Notebook, but create a new Notebook with only the assigned work to submit a clean version for marking. Please submit a generated PDF of the Notebook, which may be done using File->Print and your web browser/OS's PDF export functionality.

Start by working through the teaching activities below, exploring how DTrace and Python interact to allow us to capture and process trace data to understand a simple workload. Then create a new Notebook to answer the specific exercises at the bottom of this Notebook.

## 1. Running and tracing the workload

In these exercises, we will use the UNIX `dd(1)` command used in examples during lecture.  dd copies data from `stdin` to `stdout` in fixed size blocks.  We will use a simple invocation that copies zeros from `/dev/zero` into the null device, `/dev/null`, with a block size of 1KiB in quantity 10000.

`dd if=/dev/zero of=/dev/null bs=1k count=10000`

### Running dd(1) from Jupyter

We can use Jupyter's `!` syntax to run UNIX commands; we will run it without `status=none` so that a message is printed when it completes:

In [ ]:
!dd if=/dev/zero of=/dev/null bs=1k count=10000

### Running the dtrace(1) command-line tool from Jupyter

We can use the DTrace command-line tool via the same syntax. This example counts system calls when running our `dd(1)` command. Note that we use `execname` to filter probe firings even though `-s` is specified, in order to avoid tracing concurrent system activity, including from DTrace and Python.

In [ ]:
!dtrace -n 'syscall:::entry /execname == "dd"/ { @syscalls[probefunc] = count(); }' -c 'dd if=/dev/zero of=/dev/null bs=1k count=10000'

## 2. Capturing DTrace output in Python

To better analyse and present DTrace output, we use the `python-dtrace` module to run DTrace and capture its output directly to Python data structures.

## Import the DTrace module

First import the `python-dtrace` module. This must be done once per session:

In [ ]:
from dtrace import DTraceConsumerThread
import subprocess

## Define a DTrace convenience function

Next, abstract a little away from the mechanics of the DTrace module. Callers will need to provide a D script, a "walker" function to handle data inputs, and a command line to execute. This must be done once per session:

In [ ]:
def dtrace_synchronous(script, walker, cmdline):
    """
    script - D script
    walker - Walker routine to receive data
    cmdline - Command to run
    """
    
    # Create a seperate thread to run the DTrace instrumentation
    dtrace_thread = DTraceConsumerThread(script,
                                     walk_func=walker,
                                     out_func=lambda v: None,
                                     chew_func=lambda v: None,
                                     chewrec_func=lambda v: None,
                                     sleep=1)
    
    # Start the DTrace instrumentation
    dtrace_thread.start()

    # Display header to indicate that dd(1) has started
    print("## Starting ", cmdline)

    output_dtrace = subprocess.run(cmdline.split(" "))
        
    # The benchmark has completed - stop the DTrace instrumentation
    dtrace_thread.stop()
    dtrace_thread.join()

    # Display footer to indicate that the benchmarking has finished
    print("## Finished ", cmdline)
    
    # Explicitly free DTrace resources.
    # Python's Garbage Collector would free DTrace resources when
    # dtrace_thread is reassigned, e.g. when the cell is reexecuted.
    # This could be confusing when analysing kernel from a terminal
    # and the notebook at the same time.
    del dtrace_thread

## Collect system-call counts

We run the same script as above, counting system calls by name, only this time capture the data using the `python-dtrace` module directly into Python data structures.  Keys passed to `syscall_count_walker` are system-call names, and values are counts:

In [ ]:
# Sample D-language script embedded in a Python string.
syscall_count_script = """
syscall:::entry /execname == "dd"/ { @syscalls[probefunc] = count(); }
"""

from collections import defaultdict
syscall_count_values = defaultdict(int)
    
def syscall_count_walker(action, identifier, keys, value):
    """
    action -- type of action (sum, avg, ...)
    identifier -- the id.
    keys -- list of keys.
    value -- the value.
    """
    syscall_count_values[keys[0]] += value

# Run it all under DTrace
dtrace_synchronous(syscall_count_script, syscall_count_walker, "dd if=/dev/zero of=/dev/null bs=1k count=10000")

# Print the syscalls and their frequency
for x in syscall_count_values.keys():
    print("Number of ", x, " calls: ", syscall_count_values[x])


This approach can be used to extract a variety of kernel trace data using DTrace. One known limitation is that stack() results are stored as a set of code addresses, rather than being expanded to strings, so you will likely prefer to use the `dtrace(1)` command-line tool to capture stack data.

## Plotting a system-call count histogram

Results from `python-dtrace` can be (relatively) easily plotted using Python's standard plotting modules, such as `matplotlib`:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

labels = syscall_count_values.keys()
values = syscall_count_values.values()

y_pos = np.arange(len(labels))
fig, ax = plt.subplots()
hbars = ax.barh(y_pos, values)
ax.set_yticks(y_pos)
ax.set_yticklabels(labels)
ax.invert_yaxis()
ax.bar_label(hbars, fmt='%d')
ax.set_xlabel('Score')
ax.set_title('Bar plot')
fig.set_size_inches(12,6)

plt.show()


## Collect timer-driven profiling data

Now we repeat the same `dd(1)` workload, but with timer-driving profiling using DTrace's `profile-4997` probe.  In the prior script, keys were simple scalars. Now they are arrays containing stack frames. Values continue to contain counts.

It is possible that you will see an empty key; this occurs when the profiling probe fires while code is executing in userspace, and so there is no current kernel stack trace.

In [ ]:
# Sample D-language script embedded in a Python string.
profile_script = """
profile:::profile-4997 /execname == "dd"/ { @traces[stack()] = count(); }
"""

from collections import defaultdict
profile_values = defaultdict(int)

def profile_walker(action, identifier, keys, value):
    """
    action -- type of action (sum, avg, ...)
    identifier -- the id.
    keys -- list of keys.
    value -- the value.
    """
    print(keys, value)
    # We have to create a tuple because lists are not hashable
    hashable_key = tuple(keys)
    if keys == []:
        hashable_key = tuple(["userspace"])
    profile_values[hashable_key] += value

# Run it all under DTrace
dtrace_synchronous(profile_script, profile_walker, "dd if=/dev/zero of=/dev/null bs=1k count=10000")

# Print the stack traces and their frequency
for x in profile_values.keys():
    print(f'Number of {x} calls: {profile_values[x]}')


## Plotting a flame graph

For backtrace data derived from profiling, a flamegraph can be invaluable.
The original flamegraph implementation is in Perl, but we have a subset available via a new `pyflamegraph` module developed locally:

In [ ]:
import pyflamegraph

trace = pyflamegraph.TraceElement.from_pydtrace_dict(profile_values)

stack_tree = pyflamegraph.StackTree.from_trace(trace)

# Some sample transformations you can do
# stack_tree = stack_tree.filter(lambda s: s.num_points > 2)
# stack_tree = stack_tree.zoom("sys_read")

# You can also print the StackTree for a text-based analysis
# print(stack_tree)

flame_graph = pyflamegraph.Flamegraph.from_stack_tree(stack_tree)

# Copied from https://stackoverflow.com/a/36368116/11751242 to make plot larger
# You'll almost certainly have to mess with this on other monitors
fig, ax = plt.subplots(
    figsize=(16,8), 
    dpi= 200,
)

flame_graph.plot(fig, ax)

plt.show()

    